In [1]:
import torchio as tio
from glob import glob
from tqdm import tqdm
import SimpleITK as sitk
import os

In [2]:
for patient in tqdm(glob("/home/johannes/Data/SSD_2.0TB/GNN_pCR/data/breast_cancer/data_processed/ISPY2*")):
    # print(patient) 

    dce_files = sorted(glob(f"{patient}/*DCE_timepoint*.nii.gz"))
    ftv_files = sorted(glob(f"{patient}/*Analysis*.nii.gz"))

    for i in range(len(dce_files)//3):

        seg = sitk.ReadImage(ftv_files[i], sitk.sitkUInt16)
        ftv = sitk.Cast((seg == 1), sitk.sitkUInt8)
        sitk.WriteImage(ftv, "./tmp_ftv.nii.gz")
        bounding_box = sitk.Cast((seg == 1) | (seg == 17), sitk.sitkUInt8)
        sitk.WriteImage(bounding_box, "./tmp_bounding_box.nii.gz")

        dce_tp0=tio.ScalarImage(dce_files[i*3])
        dce_tp1=tio.ScalarImage(dce_files[i*3+1])
        dce_tp2=tio.ScalarImage(dce_files[i*3+2])
        ftv=tio.LabelMap("./tmp_ftv.nii.gz")
        bounding_box=tio.LabelMap("./tmp_bounding_box.nii.gz")

        dce_tp0 = tio.ToCanonical()(dce_tp0)
        dce_tp1 = tio.ToCanonical()(dce_tp1)
        dce_tp2 = tio.ToCanonical()(dce_tp2)
        
        ftv = tio.ToCanonical()(ftv)
        bounding_box = tio.ToCanonical()(bounding_box)
        
        ftv = tio.Resample(target=dce_tp0)(ftv)
        bounding_box = tio.Resample(target=dce_tp0)(bounding_box)

        subject = tio.Subject(
            dce_tp0=dce_tp0,
            dce_tp1=dce_tp1,
            dce_tp2=dce_tp2,
            ftv=ftv,
            bounding_box=bounding_box,
        )

        transform = tio.Resample((2.0, 2.0, 2.0))
        subject_resampled = transform(subject)

        transform = tio.CropOrPad(target_shape=(64, 64, 64), mask_name='bounding_box')
        subject_cropped = transform(subject_resampled)

        transform = tio.ZNormalization()
        subject_cropped["dce_tp0"] = transform(subject_cropped["dce_tp0"])
        subject_cropped["dce_tp1"] = transform(subject_cropped["dce_tp1"])
        subject_cropped["dce_tp2"] = transform(subject_cropped["dce_tp2"])

        subject_cropped["dce_tp0"].save(dce_files[i*3].replace(".nii.gz", "_resampled_cropped.nii.gz"))
        subject_cropped["dce_tp1"].save(dce_files[i*3+1].replace(".nii.gz", "_resampled_cropped.nii.gz"))
        subject_cropped["dce_tp2"].save(dce_files[i*3+2].replace(".nii.gz", "_resampled_cropped.nii.gz"))
        subject_cropped["ftv"].save(ftv_files[i].replace(".nii.gz", "_resampled_cropped.nii.gz"))        

        os.remove("./tmp_ftv.nii.gz")
        os.remove("./tmp_bounding_box.nii.gz")

  0%|          | 3/625 [00:06<24:02,  2.32s/it]/home/johannes/Data/SSD_2.0TB/GNN_pCR/conda_env/lib/python3.11/site-packages/torchio/transforms/preprocessing/spatial/crop_or_pad.py:298: RuntimeWarning: All values found in the mask "bounding_box" are zero. Using volume center instead
  padding_params, cropping_params = self.compute_crop_or_pad(subject)
100%|██████████| 625/625 [33:24<00:00,  3.21s/it] 
